<a href="https://colab.research.google.com/github/dmeoli/NeuroSAT/blob/master/GATQSAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/My Drive

/content/gdrive/My Drive


In [3]:
import os

if not os.path.isdir('neuroSAT'):
  !git clone --recurse-submodules https://github.com/dmeoli/neuroSAT
  %cd neuroSAT
else:
  %cd neuroSAT
  !git pull

/content/gdrive/My Drive/neuroSAT
Already up to date.


In [4]:
!pip install -r requirements.txt

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html, https://data.pyg.org/whl/torch-1.10.0+cu113.html
     |████████████████████████████████| 124 kB 7.5 MB/s 
     |██████████████▋                 | 834.1 MB 1.2 MB/s eta 0:13:18tcmalloc: large alloc 1147494400 bytes == 0x5647b05b0000 @  0x7ff7a9b04615 0x56474a56c4cc 0x56474a64c47a 0x56474a56f2ed 0x56474a660e1d 0x56474a5e2e99 0x56474a5dd9ee 0x56474a570bda 0x56474a5e2d00 0x56474a5dd9ee 0x56474a570bda 0x56474a5df737 0x56474a661c66 0x56474a5dedaf 0x56474a661c66 0x56474a5dedaf 0x56474a661c66 0x56474a5dedaf 0x56474a571039 0x56474a5b4409 0x56474a56fc52 0x56474a5e2c25 0x56474a5dd9ee 0x56474a570bda 0x56474a5df737 0x56474a5dd9ee 0x56474a570bda 0x56474a5de915 0x56474a570afa 0x56474a5dec0d 0x56474a5dd9ee
     |██████████████████▌             | 1055.7 MB 1.3 MB/s eta 0:09:51tcmalloc: large alloc 1434370048 bytes == 0x56474ddd8000 @  0x7ff7a9b04615 0x56474a56c4cc 0x56474a64c47a 0x56474a56f2ed 0x56474a660e1d 0x56474a5e2e99 0x56

In [5]:
datasets = {'uniform-random-3-sat': {'train': ['uf50-218', 'uuf50-218',
                                               'uf100-430', 'uuf100-430'],
                                     'val': ['uf50-218', 'uuf50-218',
                                             'uf100-430', 'uuf100-430'],
                                     'inner_test': ['uf50-218', 'uuf50-218',
                                                    'uf100-430', 'uuf100-430'],
                                     'test': ['uf250-1065', 'uuf250-1065']},
            'graph-coloring': {'train': ['flat50-115'],
                               'val': ['flat75-180'],
                               'test': ['flat30-60',
                                        'flat100-239',
                                        'flat125-301',
                                        'flat150-360',
                                        'flat175-417',
                                        'flat200-479']}}

# GATQSAT

In [6]:
%cd GraphQSAT

/content/gdrive/My Drive/neuroSAT/GraphQSAT


### Build C++

In [7]:
%cd minisat
!sudo ln -s --force /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy /usr/include/numpy  # https://stackoverflow.com/a/44935933/5555994
!make distclean && CXXFLAGS=-w make && make python-wrap PYTHON=python3.7
!apt install swig
!swig -fastdispatch -c++ -python minisat/gym/GymSolver.i
%cd ..

/content/gdrive/My Drive/neuroSAT/GraphQSAT/minisat
rm -f   build/release/minisat/core/Solver.o  build/release/minisat/core/Main.o  build/release/minisat/simp/SimpSolver.o  build/release/minisat/simp/Main.o  build/release/minisat/utils/System.o  build/release/minisat/utils/Options.o  build/release/minisat/gym/GymSolver.o   build/debug/minisat/core/Solver.o  build/debug/minisat/core/Main.o  build/debug/minisat/simp/SimpSolver.o  build/debug/minisat/simp/Main.o  build/debug/minisat/utils/System.o  build/debug/minisat/utils/Options.o  build/debug/minisat/gym/GymSolver.o   build/profile/minisat/core/Solver.o  build/profile/minisat/core/Main.o  build/profile/minisat/simp/SimpSolver.o  build/profile/minisat/simp/Main.o  build/profile/minisat/utils/System.o  build/profile/minisat/utils/Options.o  build/profile/minisat/gym/GymSolver.o   build/dynamic/minisat/core/Solver.o  build/dynamic/minisat/core/Main.o  build/dynamic/minisat/simp/SimpSolver.o  build/dynamic/minisat/simp/Main.o  build/dynam

## Graph Coloring

Graph coloring benchmarks have only 100 problems each, except for *flat50-115* which contains 1000, so we do not split them into training/validation/test sets using *flat50-115* for training and *flat75-180* to do model selection.

In [8]:
PROBLEM_TYPE='graph-coloring'

### Add metadata for evaluation (train and validation set)

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['train'] + datasets[PROBLEM_TYPE]['val']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME"

[Errno 2] No such file or directory: '/content/gdrive/My Drive/neuro-sat/data/graph-coloring/flat50-115/METADATA'
No metadata available, that is fine for metadata generator.
Testing agent <MiniSAT Agent>... with_restarts is set to False
Rollout 1, steps 60, num_restarts 0.
Rollout 2, steps 13, num_restarts 0.
Rollout 3, steps 15, num_restarts 0.
Rollout 4, steps 10, num_restarts 0.
Rollout 5, steps 19, num_restarts 0.
Rollout 6, steps 18, num_restarts 0.
Rollout 7, steps 14, num_restarts 0.
Rollout 8, steps 7, num_restarts 0.
Rollout 9, steps 18, num_restarts 0.
Rollout 10, steps 12, num_restarts 0.
Rollout 11, steps 13, num_restarts 0.
Rollout 12, steps 18, num_restarts 0.
Rollout 13, steps 23, num_restarts 0.
Rollout 14, steps 31, num_restarts 0.
Rollout 15, steps 28, num_restarts 0.
Rollout 16, steps 26, num_restarts 0.
Rollout 17, steps 12, num_restarts 0.
Rollout 18, steps 13, num_restarts 0.
Rollout 19, steps 18, num_restarts 0.
Rollout 20, steps 17, num_restarts 0.
Rollout 21, s

### Train

In [9]:
for TRAIN_PROBLEM_NAME, VAL_PROBLEM_NAME in zip(datasets[PROBLEM_TYPE]['train'],
                                                datasets[PROBLEM_TYPE]['val']):
  !python dqn.py \
    --logdir log \
    --env-name sat-v0 \
    --train-problems-paths ../data/"$PROBLEM_TYPE"/"$TRAIN_PROBLEM_NAME" \
    --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$VAL_PROBLEM_NAME" \
    --lr 0.00002 \
    --bsize 64 \
    --buffer-size 20000 \
    --eps-init 1.0 \
    --eps-final 0.01 \
    --gamma 0.99 \
    --eps-decay-steps 30000 \
    --batch-updates 50000 \
    --history-len 1 \
    --init-exploration-steps 5000 \
    --step-freq 4 \
    --target-update-freq 10 \
    --loss mse \
    --opt adam \
    --save-freq 500 \
    --grad_clip 0.1 \
    --grad_clip_norm_type 2 \
    --eval-freq 1000 \
    --eval-time-limit 3600 \
    --core-steps 4 \
    --expert-exploration-prob 0.0 \
    --priority_alpha 0.5 \
    --priority_beta 0.5 \
    --e2v-aggregator sum \
    --n_hidden 1 \
    --hidden_size 64 \
    --decoder_v_out_size 32 \
    --decoder_e_out_size 1 \
    --decoder_g_out_size 1 \
    --encoder_v_out_size 32 \
    --encoder_e_out_size 32 \
    --encoder_g_out_size 32 \
    --core_v_out_size 64 \
    --core_e_out_size 64 \
    --core_g_out_size 32 \
    --activation relu \
    --penalty_size 0.1 \
    --train_time_max_decisions_allowed 500 \
    --test_time_max_decisions_allowed 500 \
    --no_max_cap_fill_buffer \
    --lr_scheduler_gamma 1 \
    --lr_scheduler_frequency 3000 \
    --independent_block_layers 0 \
    --use_attention \
    --heads 3

Streaming output truncated to the last 5000 lines.
Episode 18852: Return -0.4.
Episode 18853: Return -0.4.
Episode 18854: Return -0.5.
Episode 18855: Return -0.4.
Episode 18856: Return -0.4.
Episode 18857: Return -0.5.
Episode 18858: Return -0.5.
Episode 18859: Return -0.6.
Episode 18860: Return -0.7999999999999999.
Episode 18861: Return -0.8999999999999999.
Episode 18862: Return -0.7999999999999999.
Episode 18863: Return -0.7.
Episode 18864: Return -0.4.
Episode 18865: Return -0.9999999999999999.
Episode 18866: Return -0.5.
Episode 18867: Return -0.8999999999999999.
Episode 18868: Return -0.2.
Episode 18869: Return -0.7.
Episode 18870: Return -0.5.
Episode 18871: Return -0.30000000000000004.
Episode 18872: Return -0.5.
Episode 18873: Return -0.7.
Episode 18874: Return -0.4.
Episode 18875: Return -0.5.
Episode 18876: Return -0.7.
Episode 18877: Return -0.30000000000000004.
Episode 18878: Return -0.2.
Episode 18879: Return -1.4000000000000001.
Episode 18880: Return -0.5.
Episode 18881: 

### Add metadata for evaluation (test set)

In [ ]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test']:
  !python add_metadata.py --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME"

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/neuro-sat/data/uniform-random-3-sat/test/uf50-218/METADATA'
No metadata available, that is fine for metadata generator.
Testing agent <MiniSAT Agent>... with_restarts is set to False
Rollout 1, steps 38, num_restarts 0.
Rollout 2, steps 16, num_restarts 0.
Rollout 3, steps 8, num_restarts 0.
Rollout 4, steps 15, num_restarts 0.
Rollout 5, steps 87, num_restarts 0.
Rollout 6, steps 17, num_restarts 0.
Rollout 7, steps 13, num_restarts 0.
Rollout 8, steps 27, num_restarts 0.
Rollout 9, steps 29, num_restarts 0.
Rollout 10, steps 28, num_restarts 0.
Rollout 11, steps 17, num_restarts 0.
Rollout 12, steps 40, num_restarts 0.
Rollout 13, steps 67, num_restarts 0.
Rollout 14, steps 17, num_restarts 0.
Rollout 15, steps 66, num_restarts 0.
Rollout 16, steps 62, num_restarts 0.
Rollout 17, steps 38, num_restarts 0.
Rollout 18, steps 16, num_restarts 0.
Rollout 19, steps 32, num_restarts 0.
Rollout 20, steps 94, num_restarts 0.
Rollo

### Evaluate

In [10]:
MODEL_DIR='Nov01_10-54-06_a8b07447f6a4'
CHECKPOINT='model_50000'

In [11]:
for PROBLEM_NAME in datasets[PROBLEM_TYPE]['test']:
  for MODEL_DECISION in [10, 50, 100, 300, 500, 1000]:
    !python evaluate.py \
      --logdir log \
      --env-name sat-v0 \
      --core-steps -1 \
      --eps-final 0.0 \
      --eval-time-limit 100000000000000 \
      --no_restarts \
      --test_time_max_decisions_allowed "$MODEL_DECISION" \
      --eval-problems-paths ../data/"$PROBLEM_TYPE"/"$PROBLEM_NAME" \
      --model-dir runs/"$MODEL_DIR" \
      --model-checkpoint "$CHECKPOINT".chkp \
      >> runs/"$MODEL_DIR"/"$PROBLEM_NAME"-gatqsat-max"$MODEL_DECISION".tsv

In [ ]:
%cd ..

/content/gdrive/My Drive/neuro-sat
